# Reto 3: El problema final

Este reto consiste en predecir si una imagen contiene o no signos de retinopatía diabética (DR).
Para ello se proporciona un conjunto de datos que contiene características extraídas de imágenes.
Las características extraídas son:
+ [0] Evaluación de la calidad de la imagen , donde 0 = mala calidad 1 = calidad suficiente.
+ [1] El resultado binario de la evaluación previa, donde 1 indica una anormalidad retiniana severa y 0 su falta.
+ [2-7] Los resultados de la detección de microaneurismas (MA). Cada valor de característica representa el número de MAs encontrados en los niveles de confianza alfa = 0.5, ... , 1, respectivamente.
+ [8-15] contienen la misma información que [2-7] para los exudados. <br>
Sin embargo, ya que los exudados están representados por un conjunto de puntos en lugar del número de
píxeles de las lesiones, estas características se normalizan dividiendo los número de lesiones por  el diámetro de la ROI para compensar las diferentes tamaños de las imágenes.
+ [16] La distancia euclidiana del centro dela mácula y el centro del disco óptico para proporcionar información importante con respecto a la condición del paciente. Esta característica también se normaliza con el diámetro de la ROI
+ [17] El diámetro del disco óptico.
+ [18] El resultado binario de la clasificación basada en AM/FM.
+ [19] Etiqueta de clase. 1 = contiene signos de DR

El conjunto de datos, *retinopatia_reto3.csv*, consiste en 592 muestras de imágenes; cada uno de ellos representado por 19 características.

Para evaluar las propuestas se utilizará un conjunto de datos que se mantendrá oculto hasta después de la entrega

### Requisitos
+ **Se debe utilizar algún tipo de reducción de la dimensión o aprendizaje de variedades**
+ A continuación se debe entrenar un clasificador o combinación de clasificadores con las características transformadas según el requisito anterior.
+ Se debe entregar un cuaderno Jupyter con el nombre de los participantes.<br>
  *Por ejemplo*:   **Cuesta_Hinton.ipynb**
+ El cuaderno entregado debe seguir la estructura y reglas de este cuaderno

### Competición
+ Todos los cuadernos entregados se subirán al repo de GitHub y se ejecutarán en Binder, donde ya estará en conjunto de test que estaba oculto.
+ El resultado que se obtenga será la puntuación del reto.
+ **Importante** Es muy fácil asegurarte de que tu código funciona:
    1. Agrupa todo tu código en una única celda
    2. Copialo en el cuaderno del reto que hay en Binder
    3. Ejecuta el cuaderno 
    
### Plazo: lunes 16 de nov. a las 6 am.
Es decir, incluye toda la noche del domingo 15 de nov.
 
> "The final problem", A. C. Doyle (Strand Magazine, diciembre 1983), 
es el relato corto en el que Sherlock Holmes se enfrenta al Prof. Moriarty en las cataratas Reichenbach.

---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es

In [ ]:
# NO TOCAR ESTA CELDA
# Conjunto distribuido para el reto

Challange_filename = '../../Datasets/retinopatia_reto3.csv'

In [ ]:
# NO TOCAR ESTA CELDA
# El conjunto de test cambiará una vez se cierre la entrega
# Ahora mismo es el mismo conjunto que el de entrenamiento

Test_filename = '../../Datasets/retinopatia_test.csv' #<-- este nombre cambiará después del plazo de entrega

In [3]:
# NO TOCAR ESTA CELDA

#-[1]. Load data from CSV and put all in a single dataframe 'FullSet'

import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append('../../MyUtils/')
import MyUtils as my
seed = 1234 #<- random generator seed (comment to get randomness)

#-[2]. Load data from CSV and put all in a single dataframe 'FullSet'

FullSet = pd.read_csv(Challange_filename, header=0)
FullX = FullSet.iloc[:,:-1]
FullY = FullSet.iloc[:,-1]

<table style="width:100%;"> 
 <tr style='background:lime'>
  <td style="text-align:left">
      <h2>Tu código debe empezar a partir de aquí y puede tener tantas celdas como quieras</h2>
      <p> Si quieres, puedes borrar (o convertir en RawNBConvert) las celdas de ejemplo
      <h3>Importante:</h3>
      <p>Tu código debe producir las siguientes variables: </p>
      <p> $\quad \bullet$ <b>clf:</b> el clasificador final con el que se realizará el test<br>
       $\quad \bullet$ <b>X_test:</b> el conjunto de test listo para ser usado por el método <b>predict</b><br>
       $\quad \bullet$ <b>Y_test:</b> es el vector de etiquetas del conjunto de X_test listo para ser usado por el método <b>confusion_matrix</b>
      </p>
  </td>
 </tr>
</table>

In [4]:
nombres = ['Pablo Asensio', 'Ruben Oliver']

In [5]:
# para crear diferentes clasificadores con unas muestras similares
from sklearn.model_selection import StratifiedKFold
N = 7 # numero de clasificadores que se van a entrenar
splitter = StratifiedKFold(n_splits=N)

# para Booting con AdaBoost
from sklearn.ensemble import AdaBoostClassifier

<table style="width:100%;"> 
 <tr style='background:orange'>
  <td style="text-align:left">
      <h2>(Mensaje a posteriori) Se elige un Decision Tree con 3 niveles: Se anotan los resultados promedios al lado</h2>
  </td>
 </tr>
</table>

In [6]:
# tipos de clasificadores que se prueban
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

lin_clf = SVC(kernel="linear", C=1, probability=True)
tree_clf = DecisionTreeClassifier(
    max_depth=3
)  # 1 -> 0.73,  2 -> 0.61, 3 -> 0.92, 4 -> [0.64, 0.93] (no estable)??
nbc_clf = GaussianNB()

In [7]:
# variables de almacenamiento de los clasificadores y similares
clfs = []
scalers = []
pcas = []
precision = 0 # inicializada en 0
seed = 1234

<table style="width:100%;"> 
 <tr style='background:orange'>
  <td style="text-align:left">
      <h2>Bucle que entrena los N clasificadores</h2>
  </td>
 </tr>
</table>

In [8]:
from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

for train_ix, test_ix in splitter.split(FullX, FullY):
    # se obtienen los diferentes conjuntos de datos para el entrenamiento
    X_train_split = FullX.loc[train_ix].reset_index(drop=True)
    Y_train_split = FullY.loc[train_ix].reset_index(drop=True)
    X_test_split = FullX.loc[test_ix].reset_index(drop=True)
    Y_test_split = FullY.loc[test_ix].reset_index(drop=True)
    
    # se escalan los datos de entrenamiento
    scaler = MinMaxScaler(feature_range=(0, 100))
    X = scaler.fit_transform(X_train_split)
    Y = Y_train_split.values.ravel()
    scalers.append(scaler)
    
    # se aplica una reduccion de datos a un 85% (A posteriori: con 0.75 y 0.8 no era estable (0.6-0.9)??)
    n_components = 0.85
    if n_components <= X.shape[1]:
        pca = PCA(n_components=n_components)
        pca.fit(X)
        X_proy = pca.transform(X)
    else:
        print(
            "ERROR: the number of princial components has to be less or equal than data dimension !"
        )
    
    #IMPORTANTE ABAJO
    """ -----> SE HA PROBADO CON n_components = 19 o inferior y no se obtenian buenos resultados... 
        -----> Con n_components = 100 si, por se descarta por no reducir dimensiones
    from sklearn.decomposition import KernelPCA
    
    n_components = 100  # <- now, there is NO limit in the number of "components"

    kernel = "rbf"  # options are: "linear", "poly", "rbf", "sigmoid"
    kernel_parameter = 1

    pca = KernelPCA(n_components=n_components,
                        kernel=kernel, gamma=kernel_parameter, fit_inverse_transform=True)

    X_proy = pca.fit_transform(X)
    """
    pcas.append(pca)
    
    n_estimators = 500
    learning_rate = 0.4
    choice_clf = "DT"  # 'LinearSVC', 'NBC' , 'DT' # mejor resultado DT
    
    if choice_clf == "LinearSVC":
        clf = AdaBoostClassifier(
            lin_clf,
            n_estimators=n_estimators,
            algorithm="SAMME.R",
            learning_rate=learning_rate,
        )
    elif choice_clf == "DT":
        clf = AdaBoostClassifier(
            tree_clf,
            n_estimators=n_estimators,
            algorithm="SAMME.R",
            learning_rate=learning_rate,
        )
    elif choice_clf == "NBC":
        clf = AdaBoostClassifier(
            nbc_clf,
            n_estimators=n_estimators,
            algorithm="SAMME.R",
            learning_rate=learning_rate,
        )
    else:
        print("Choose one base classifier")
        
    # Se entrena el clasificador con los datos proyentados
    clf.fit(X_proy, Y)

    X_test = scaler.transform(X_test_split)
    Y_test = Y_test_split.values.ravel()
    
    X_test = pca.transform(X_test)
    Y_hat = clf.predict(X_test)
    
    conf_mat = confusion_matrix(Y_test, Y_hat)
    
    N_success = np.trace(conf_mat)
    N_fails = Y_test.shape[0] - N_success

    # Se guarda el clasificador con la puntuacion obtenida
    clfs.append((clf, 100 * N_success / (N_success + N_fails)))
    precision += 100 * N_success / (N_success + N_fails)

<table style="width:100%;"> 
 <tr style='background:orange'>
  <td style="text-align:left">
      <h2>Se elige elige el clasificador que mas se ajusta a la media de los clasificadores</h2>
  </td>
 </tr>
</table>

In [9]:
mean = precision / N

scaler_count = 0
clf_aux = clfs[0]
scaler = scalers[0]
pca = pcas[0]
for classifier in clfs:
    #compara las puntuaciones
    if abs(clf_aux[1] - mean) > abs(classifier[1] - mean):
        clf_aux = classifier
        scaler = scalers[scaler_count]
        pca = pcas[scaler_count]
    scaler_count += 1

clf = clf_aux[0]

In [10]:
# -- ejemplo de test --

FullSet = pd.read_csv(Test_filename, header=0)
TestX = FullSet.iloc[:, :-1]
TestY = FullSet.iloc[:, -1]
X_test = scaler.transform(TestX)
Y_test = TestY.values.ravel()
X_test = pca.transform(X_test)

<table style="width:100%;"> 
 <tr style='background:pink'>
  <td style="text-align:left">
      <h2>A partir de aquí ya no se pueden modificar las celdas</h2>
          <h3>Comprueba que:</h3>
          <p> $\quad \bullet$ tu clasificador está almacenado en la variable <b>clf</b><br>
              $\quad \bullet$ tienes el conjunto de test correctamente almacenado en la variable <b>X_test</b><br>
              $\quad \bullet$ tienes las etiquetas del conjunto de test correctamente almacenadas en la variable <b>Y_test</b><br>
          </p>
      
  </td>
 </tr>
</table>

## Test

In [11]:
# NO TOCAR ESTA CELDA

from sklearn.metrics import confusion_matrix

Y_hat = clf.predict(X_test)
conf_mat = confusion_matrix(Y_test , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_test.shape[0]-N_success
#-------------------------------
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)

['Pablo Asensio', 'Ruben Oliver'] 

Confusion matrix:

[[421  25]
 [ 33 472]] 

Outcome:

  :) HIT  = 893, (93.90%)
  :( FAIL = 58, (6.10%)
